### Read SLHA files and store the data in a pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
pd.options.mode.chained_assignment = None #Disable copy warnings

/tmp/ipykernel_58856/1618904452.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import glob,imp,os,sys


In [2]:
slhaFolder = '../slhaFiles/sms'
# slhaFolder = '../slhaFiles/dmeft'

In [3]:
#Get SLHA data:
slhaData = []
for f in glob.glob(os.path.join(slhaFolder,'*.slha')):
    try:
        slha = pyslha.readSLHAFile(f)
    except:
        continue
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key > 25])
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key > 25])
    parsDict = dict([[str(key),abs(val)] for key,val in slha.blocks['FRBLOCK'].items()])
    BRsDict = {}
    for pdg,val in slha.decays.items():
        if not abs(pdg) > 100000:
            continue
        initialState = str(pdg)
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([str(pid) for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 
                'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict, 'pars' : parsDict}
    slhaData.append(slhaDict)

In [4]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)

In [5]:
#Save DataFrame to pickle file:
slhaDF.to_pickle(os.path.join(os.path.dirname(slhaFolder),os.path.basename(slhaFolder)+'_slha.pcl'))

In [6]:
print(len(slhaDF))

206
